In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path
import fastai
from fastai.vision import *
import cv2
import torch
from torch.nn import Conv2d
from sklearn.model_selection import train_test_split
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
path = Path('../input/recursion-cellular-image-classification')

In [ ]:
tr_df = pd.read_csv(path/'train.csv')

In [ ]:
tr_df['site'] = 1
tr_df_copy = tr_df.copy()
tr_df_copy['site'] = 2
tr_df = tr_df.append(tr_df_copy, ignore_index=True)

In [ ]:
tr_df.columns

In [ ]:
def get_img_path(experiment, plate, well, site=1):
    return experiment + '/' + 'Plate' + str(plate) + '/' + well + '_s{}'.format(site)

In [ ]:
tr_df['path'] = tr_df.apply(lambda row : get_img_path(row.experiment, row.plate, row.well, row.site), axis=1)

In [ ]:
tr_df['cellline'] = tr_df.apply(lambda row : row.experiment[:-3], axis=1)

In [ ]:
cell_lines = list(tr_df.cellline.unique())

In [ ]:
cell_lines

In [ ]:
class RecursionImageList(ImageList):
    def open(self, fn:PathOrStr) -> Image:
        fn = str(fn)
        imgs = []
        for channel in [1, 2, 3, 4, 5, 6]:
            img_path = fn + '_w{}'.format(channel) + '.png'
            imgs.append(cv2.imread(img_path, cv2.IMREAD_GRAYSCALE))
        img = cv2.merge(imgs)
        img = img / 255.0
        return Image(px=pil2tensor(img, np.float32))

In [ ]:
# tr_df_small = tr_df.sample(n=120, random_state=42)

In [ ]:
src = (
    RecursionImageList.from_df(tr_df, path=path/'train', cols=['path'])
)

In [ ]:
df = tr_df
tr_df, va_df = train_test_split(df, test_size=0.05, random_state=42, stratify = df.cellline)
src = src.split_by_list(RecursionImageList.from_df(tr_df, path=path/'train', cols=['path']), 
                        RecursionImageList.from_df(va_df, path=path/'train', cols=['path']))

In [ ]:
# src = src.split_subsets(train_size=0.001, valid_size=0.001)

In [ ]:
src = src.label_from_df(cols=['cellline'])

In [ ]:
data = src.transform(size=512).databunch(bs=16, pin_memory=True)

In [ ]:
data

In [ ]:
precision = Precision(average="macro")
recall = Recall(average="macro")
loss_func = CrossEntropyFlat()

In [ ]:
learn = cnn_learner(data, models.resnet18, metrics=[accuracy, error_rate, precision, recall],
                    loss_func=loss_func,
                    model_dir='/kaggle/working', pretrained=True)

In [ ]:
learn.model[0][0]

In [ ]:
# let's make our model work with single channel
trained_kernel = learn.model[0][0].weight
new_conv = Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
with torch.no_grad():
    new_conv.weight[:,:] = torch.stack([torch.mean(trained_kernel, 1)] * 6, dim=1)
learn.model[0][0] = new_conv
learn.model.cuda();

In [ ]:
# !ls ../input/recursion-fast-ai-channel-2-4-resnet50

In [ ]:
# learn.load('../input/recursion-fast-ai-channel-2-4-resnet50/stage-05');

In [ ]:
# learn.unfreeze()

In [ ]:
learn.summary()

In [ ]:
# Find a good learning rate
learn.lr_find(start_lr=1.0e-5, end_lr=1e-1, num_it=256)
learn.recorder.plot(suggestion=True)

In [ ]:
!nvidia-smi

In [ ]:
learn.fit(8, 3e-4)

In [ ]:
learn.recorder.plot_losses()
learn.recorder.plot_metrics()
learn.recorder.plot_lr(show_moms=True)

In [ ]:
learn.save('stage-01')

In [ ]:
learn.validate(dl=learn.data.train_dl)

# Prediction

In [ ]:
te_df = pd.read_csv(path/'test.csv')
te_df['path1'] = te_df.apply(lambda row : get_img_path(row.experiment, row.plate, row.well, site=1), axis=1)
te_df['path2'] = te_df.apply(lambda row : get_img_path(row.experiment, row.plate, row.well, site=2), axis=1)
te_df['cellline'] = te_df.apply(lambda row : row.experiment[:-3], axis=1)
test_src1 = RecursionImageList.from_df(te_df, path=path/'test', cols=['path1'])
test_src2 = RecursionImageList.from_df(te_df, path=path/'test', cols=['path2'])

In [ ]:
learn.data.add_test(test_src1)
preds1, y = learn.get_preds(DatasetType.Test)

In [ ]:
learn.data.add_test(test_src2)
preds2,y = learn.get_preds(DatasetType.Test)

In [ ]:
preds = 0.5 * (preds1 + preds2)

In [ ]:
te_df['pred_cellline'] = preds.argmax(1)

In [ ]:
cell_line_dict = dict(enumerate(cell_lines))

In [ ]:
cell_line_dict

In [ ]:
1 - (te_df['pred_cellline'].apply(lambda row: cell_line_dict[row]) == te_df['cellline']).mean()